# Manufacturing Downtime Analysis

This is a project case analysis that aims to analyse the manufacturing efficiency, identify downtime performers and also operators' struggles

### LOADING AND CREATING A  DB FILE

In [5]:
#import pandas as pd
import pandas as pd
from sqlalchemy import create_engine

# Path to your Excel file
excel_file = "Manufacturing_Line_Productivity.xlsx"

# Read all sheet names into a dictionary of DataFrames
sheets_dict = pd.read_excel(excel_file, sheet_name=None)

# Check loaded sheet names
print(sheets_dict.keys())


dict_keys(['Line productivity', 'Products', 'Downtime factors', 'Line downtime'])


In [1]:
#Create an SQL db
import pandas as pd

# Example
df = pd.read_excel("Manufacturing_Line_Productivity.xlsx")  

# Check the first few rows
df.head()


,Date,Product,Batch,Operator,Start Time,End Time
0,2024-08-29,OR-600,422111,Mac,11:50:00,14:05:00
1,2024-08-29,LE-600,422112,Mac,14:05:00,15:45:00
2,2024-08-29,LE-600,422113,Mac,15:45:00,17:35:00
3,2024-08-29,LE-600,422114,Mac,17:35:00,19:15:00
4,2024-08-29,LE-600,422115,Charlie,19:15:00,20:39:00


In [3]:
## next is to load sql_ext
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
## Create an in-memory SQLite database and load your data
import sqlite3

# Create SQLite connection
conn = sqlite3.connect(':memory:')

# Load pandas DataFrame into SQL
df.to_sql('my_table', conn, index=False, if_exists='replace')


38

import pandas as pd

# Load all sheets
xls = pd.ExcelFile("Manufacturing_Line_Productivity.xlsx")
sheets = xls.sheet_names
print(sheets)


In [7]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect("manufacturing.db")

# Loop through sheets and save each as table
for sheet in sheets:
    df = pd.read_excel("Manufacturing_Line_Productivity.xlsx", sheet_name=sheet)
    # Replace spaces in sheet names with underscores for SQL safety
    table_name = sheet.replace(" ", "_")
    df.to_sql(table_name, conn, index=False, if_exists="replace")


### Connect with SQL magic in Jupyter

In [1]:
%load_ext sql
%sql sqlite:///manufacturing.db

'Connected: @manufacturing.db'

to clarify the names of the tables

In [30]:
%%sql
SELECT name 
FROM sqlite_master 
WHERE type='table';


 * sqlite:///manufacturing.db
Done.


name
Line_productivity
Products
Downtime_factors
Line_downtime


to get an overview of the manufactring. line_productivity table

In [11]:
%%sql
select
  *
from line_productivity
limit 5;

 * sqlite:///manufacturing.db
Done.


Date,Product,Batch,Operator,Start Time,End Time
2024-08-29 00:00:00,OR-600,422111,Mac,11:50:00.000000,14:05:00.000000
2024-08-29 00:00:00,LE-600,422112,Mac,14:05:00.000000,15:45:00.000000
2024-08-29 00:00:00,LE-600,422113,Mac,15:45:00.000000,17:35:00.000000
2024-08-29 00:00:00,LE-600,422114,Mac,17:35:00.000000,19:15:00.000000
2024-08-29 00:00:00,LE-600,422115,Charlie,19:15:00.000000,20:39:00.000000


In [3]:
%load_ext sql
%sql sqlite:///manufacturing.db

'Connected: @manufacturing.db'

#### EFFECIENCY ANALYSIS

We need to calculate the line of efficiency by
1. Determining the actual time taken in each batch
2. The planned time of each batch
3. the line efficiency result(planned time /actual time

In [29]:
%%sql
SELECT
    lp."Start Time",
    lp."End Time",
    ROUND(
        (
            (CASE 
                WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                THEN strftime('%s', lp."End Time") + 24*60*60
                ELSE strftime('%s', lp."End Time")
            END) 
            - strftime('%s', lp."Start Time")
        ) / 60.0,
        2
    ) AS actual_time_taken,
    p."Min batch time" AS planned_time,
    ROUND(
        (
            p."Min batch time" /
            (
                (
                    (CASE 
                        WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                        THEN strftime('%s', lp."End Time") + 24*60*60
                        ELSE strftime('%s', lp."End Time")
                    END)
                    - strftime('%s', lp."Start Time")
                ) / 60.0
            )
        ) * 100,
        2
    ) AS line_efficiency_percentage
FROM line_productivity lp
JOIN products p
    ON lp."Product" = p."Product";


 * sqlite:///manufacturing.db
Done.


Start Time,End Time,actual_time_taken,planned_time,line_efficiency_percentage
11:50:00.000000,14:05:00.000000,135.0,60,44.44
14:05:00.000000,15:45:00.000000,100.0,60,60.0
15:45:00.000000,17:35:00.000000,110.0,60,54.55
17:35:00.000000,19:15:00.000000,100.0,60,60.0
19:15:00.000000,20:39:00.000000,84.0,60,71.43
20:39:00.000000,21:39:00.000000,60.0,60,100.0
21:39:00.000000,22:54:00.000000,75.0,60,80.0
04:05:00.000000,06:05:00.000000,120.0,60,50.0
06:05:00.000000,07:30:00.000000,85.0,60,70.59
07:30:00.000000,09:22:00.000000,112.0,60,53.57


next is to find the median and mean. In a general case of percentage, normally the median would be 50% however, this is not a general case 
as the line efficiency cannot be 0.therefore we need to find the median and mode

In [36]:
%%sql
WITH efficiency AS (
    SELECT
        lp."Start Time",
        lp."End Time",
        ROUND(
            (strftime('%s', lp."End Time") - strftime('%s', lp."Start Time")) / 60.0,
            2
        ) AS actual_time_taken,
        p."Min batch time" AS planned_time,
        ROUND(
            (p."Min batch time" / ((strftime('%s', lp."End Time") - strftime('%s', lp."Start Time")) / 60.0)) * 100,
            2
        ) AS line_efficiency_percentage
    FROM line_productivity lp
    JOIN products p
        ON lp."Product" = p."Product"
),
ordered AS (
    SELECT 
        line_efficiency_percentage,
        ROW_NUMBER() OVER (ORDER BY line_efficiency_percentage) AS rn,
        COUNT(*) OVER () AS total_rows
    FROM efficiency
),
median_candidates AS (
    SELECT line_efficiency_percentage
    FROM ordered
    WHERE rn IN ( (total_rows + 1) / 2, (total_rows + 2) / 2 )
)
SELECT AVG(line_efficiency_percentage) AS median
FROM median_candidates;


 * sqlite:///manufacturing.db
Done.


median
63.815


now lets find the mean


In [40]:
%%sql
WITH efficiency AS (
    SELECT
        ROUND(
            (p."Min batch time" / ((strftime('%s', lp."End Time") - strftime('%s', lp."Start Time")) / 60.0)) * 100,
            2
        ) AS line_efficiency_percentage
    FROM line_productivity lp
    JOIN products p
        ON lp."Product" = p."Product"
)
SELECT ROUND(AVG(line_efficiency_percentage),2) AS mean_efficiency
FROM efficiency;




 * sqlite:///manufacturing.db
Done.


mean_efficiency
64.9


The mean is different from the median, meaning that the results are slightly positive skewed.Most production line work are working around 63% however some outliers are pushing the mean upward

now i want to see each product efficiency average

In [2]:
%load_ext sql
%sql sqlite:///manufacturing.db

'Connected: @manufacturing.db'

In [10]:
%%sql
WITH batch_efficiency AS (
    SELECT
        lp."Product",
        p.Flavor,
        ROUND(
            (
                p."Min batch time" /
                (
                    (
                        (CASE 
                            WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                            THEN strftime('%s', lp."End Time") + 24*60*60
                            ELSE strftime('%s', lp."End Time")
                        END)
                        - strftime('%s', lp."Start Time")
                    ) / 60.0
                )
            ) * 100,
            2
        ) AS line_efficiency_percentage
    FROM line_productivity lp
    JOIN products p
        ON lp."Product" = p."Product"
),
ranked AS (
    SELECT
        Product,
        Flavor,
        line_efficiency_percentage,
        ROW_NUMBER() OVER (PARTITION BY Product ORDER BY line_efficiency_percentage) AS rn,
        COUNT(*) OVER (PARTITION BY Product) AS cnt
    FROM batch_efficiency
),
median_calc AS (
    SELECT
        Product,
        Flavor,
        ROUND(AVG(line_efficiency_percentage),2) AS median_efficiency
    FROM ranked
    WHERE rn IN (
        (cnt + 1) / 2,
        (cnt + 2) / 2
    )
    GROUP BY Product, Flavor
),
summary AS (
    SELECT
        Product,
        Flavor,
        COUNT(*) AS product_count,
        ROUND(AVG(line_efficiency_percentage),2) AS avg_efficiency,
        ROUND(MIN(line_efficiency_percentage),2) AS min_efficiency,
        ROUND(MAX(line_efficiency_percentage),2) AS max_efficiency
    FROM batch_efficiency
    GROUP BY Product, Flavor
)
SELECT
    s.Product,
    s.Flavor,
    s.product_count,
    m.median_efficiency,
    s.avg_efficiency,
    s.min_efficiency,
    s.max_efficiency
FROM summary s
JOIN median_calc m
    ON s.Product = m.Product AND s.Flavor = m.Flavor
ORDER BY s.Product;


 * sqlite:///manufacturing.db
Done.


Product,Flavor,product_count,median_efficiency,avg_efficiency,min_efficiency,max_efficiency
CO-2L,Cola,5,64.47,66.11,47.8,81.67
CO-600,Cola,15,70.59,67.34,45.11,100.0
DC-600,Diet Cola,4,66.07,71.67,54.55,100.0
LE-600,Lemon lime,6,65.72,71.0,54.55,100.0
OR-600,Orange,1,44.44,44.44,44.44,44.44
RB-600,Root Beer,7,63.16,64.45,48.78,89.55


Most of the product categories fall within the median efficiency line 63.815. However, the data shows instances of time overrun which are the norm within
manufacturing. The CO-600, DC-600 AND LE-600 have shown that the planned time can be hit, but only on occasion.
out of 6 product lines, only 4 have attained the 63.815 efficiency.

the Product OR-600 is at 44.44% efficiency, which is way below the efficiency median. In the calculation of time overrun, it results in 125% which is critical. 

#### OPERATOR ANALYSIS

Lets analyse the operators performance in terms of batch completed,time taken for completion,their line efficiency,operator error ,error rate,
batches per hour

In [2]:
%load_ext sql
%sql sqlite:///manufacturing.db

'Connected: @manufacturing.db'

In [42]:
%%sql
SELECT
    lp.Operator,

    COUNT(*) AS Batches_completed,

    ROUND(SUM(
        (CASE 
            WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
            THEN strftime('%s', lp."End Time") + 24*60*60
            ELSE strftime('%s', lp."End Time")
        END) - strftime('%s', lp."Start Time")
    ) / 3600.0, 2) AS actual_time_taken_hours,
    
    ROUND(
        COUNT(*) / (
            SUM(
                (CASE 
                    WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                    THEN strftime('%s', lp."End Time") + 24*60*60
                    ELSE strftime('%s', lp."End Time")
                END) - strftime('%s', lp."Start Time")
            ) / 3600.0
        ), 
    2) AS batches_per_hour
                    
FROM line_productivity lp
GROUP BY lp.Operator
order by batches_per_hour DESC ;


 * sqlite:///manufacturing.db
Done.


Operator,Batches_completed,actual_time_taken_hours,batches_per_hour
Dee,11,17.17,0.64
Dennis,8,13.67,0.59
Charlie,11,19.3,0.57
Mac,8,14.17,0.56


we need to include the operator error and the error of rate


In [20]:
%%sql
select
    *
from Line_downtime
limit 5;

 * sqlite:///manufacturing.db
Done.


Batch,Factor_1,Factor_2,Factor_3,Factor_4,Factor_5,Factor_6,Factor_7,Factor_8,Factor_9,Factor_10,Factor_11,Factor_12
422111,None,60.0,None,None,None,None,15.0,None,None,None,None,None
422112,None,20.0,None,None,None,None,None,20.0,None,None,None,None
422113,None,50.0,None,None,None,None,None,None,None,None,None,None
422114,None,None,None,25.0,None,15.0,None,None,None,None,None,None
422115,None,None,None,None,None,None,None,None,None,24.0,None,None


In [23]:
%%sql
SELECT
    Batch,
    (
        IFNULL(Factor_1, 0) +
        IFNULL(Factor_2, 0) +
        IFNULL(Factor_3, 0) +
        IFNULL(Factor_4, 0) +
        IFNULL(Factor_5, 0) +
        IFNULL(Factor_6, 0) +
        IFNULL(Factor_7, 0) +
        IFNULL(Factor_8, 0) +
        IFNULL(Factor_9, 0) +
        IFNULL(Factor_10, 0) +
        IFNULL(Factor_11, 0) +
        IFNULL(Factor_12, 0)
    ) AS total_downtime
FROM Line_downtime
LIMIT 5;


 * sqlite:///manufacturing.db
Done.


Batch,total_downtime
422111,75.0
422112,40.0
422113,50.0
422114,40.0
422115,24.0


In [49]:
%%sql
SELECT
    lp.Operator,

    COUNT(*) AS Batches_completed,

    ROUND(SUM(
        (CASE 
            WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
            THEN strftime('%s', lp."End Time") + 24*60*60
            ELSE strftime('%s', lp."End Time")
        END) 
        - strftime('%s', lp."Start Time")
    ) /3600.0, 2) AS actual_time_taken_hours,
                          
   ROUND(
        COUNT(*) / (
            SUM(
                (CASE 
                    WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                    THEN strftime('%s', lp."End Time") + 24*60*60
                    ELSE strftime('%s', lp."End Time")
                END) - strftime('%s', lp."Start Time")
            ) / 3600.0
        ), 
    2) AS batches_per_hour,
             
   round(SUM(
        IFNULL(ld.Factor_1, 0) +
        IFNULL(ld.Factor_2, 0) +
        IFNULL(ld.Factor_3, 0) +
        IFNULL(ld.Factor_4, 0) +
        IFNULL(ld.Factor_5, 0) +
        IFNULL(ld.Factor_6, 0) +
        IFNULL(ld.Factor_7, 0) +
        IFNULL(ld.Factor_8, 0) +
        IFNULL(ld.Factor_9, 0) +
        IFNULL(ld.Factor_10, 0) +
        IFNULL(ld.Factor_11, 0) +
        IFNULL(ld.Factor_12, 0)
    )/60,2) AS total_downtime_hours
        
FROM line_productivity lp
LEFT JOIN Line_downtime ld
    ON lp.Batch = ld.Batch
GROUP BY lp.Operator
ORDER BY lp.Operator;


 * sqlite:///manufacturing.db
Done.


Operator,Batches_completed,actual_time_taken_hours,batches_per_hour,total_downtime_hours
Charlie,11,19.3,0.57,6.4
Dee,11,17.17,0.64,6.17
Dennis,8,13.67,0.59,5.03
Mac,8,14.17,0.56,5.53


In [54]:
%%sql
SELECT
    lp.Operator,

    COUNT(*) AS Batches_completed,

    ROUND(SUM(
        (CASE 
            WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
            THEN strftime('%s', lp."End Time") + 24*60*60
            ELSE strftime('%s', lp."End Time")
        END) 
        - strftime('%s', lp."Start Time")
    ) /3600.0, 2) AS actual_time_taken_hours,
                          
   ROUND(
        COUNT(*) / (
            SUM(
                (CASE 
                    WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                    THEN strftime('%s', lp."End Time") + 24*60*60
                    ELSE strftime('%s', lp."End Time")
                END) - strftime('%s', lp."Start Time")
            ) / 3600.0
        ), 
    2) AS batches_per_hour,
             
   round(SUM(
        IFNULL(ld.Factor_1, 0) +
        IFNULL(ld.Factor_2, 0) +
        IFNULL(ld.Factor_3, 0) +
        IFNULL(ld.Factor_4, 0) +
        IFNULL(ld.Factor_5, 0) +
        IFNULL(ld.Factor_6, 0) +
        IFNULL(ld.Factor_7, 0) +
        IFNULL(ld.Factor_8, 0) +
        IFNULL(ld.Factor_9, 0) +
        IFNULL(ld.Factor_10, 0) +
        IFNULL(ld.Factor_11, 0) +
        IFNULL(ld.Factor_12, 0)
    )/60,2) AS total_downtime_hours,

    ROUND(
    SUM(
        IFNULL(ld.Factor_2, 0) +
        IFNULL(ld.Factor_5, 0) +
        IFNULL(ld.Factor_6, 0) +
        IFNULL(ld.Factor_8, 0) +
        IFNULL(ld.Factor_10, 0) +
        IFNULL(ld.Factor_11, 0)
    ) / 60, 2
) AS operator_downtime_hours

FROM Line_downtime ld 
LEFT JOIN line_productivity lp
    ON lp.Batch = ld.Batch
GROUP BY lp.Operator
ORDER BY lp.Operator;

 * sqlite:///manufacturing.db
Done.


Operator,Batches_completed,actual_time_taken_hours,batches_per_hour,total_downtime_hours,operator_downtime_hours
Charlie,11,19.3,0.57,6.4,3.8
Dee,11,17.17,0.64,6.17,3.2
Dennis,8,13.67,0.59,5.03,2.73
Mac,8,14.17,0.56,5.53,3.2


now we need no-operator_downtime hours and % of operator hours .if we use the previos query it will be complex will save this as CTE

In [62]:
%%sql
WITH operator_performance_analysis AS (
    SELECT
        lp.Operator,
        COUNT(*) AS Batches_completed,
        ROUND(SUM(
            (CASE 
                WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                THEN strftime('%s', lp."End Time") + 24*60*60
                ELSE strftime('%s', lp."End Time")
            END) - strftime('%s', lp."Start Time")
        ) / 3600.0, 2) AS actual_time_taken_hours,
        ROUND(
            COUNT(*) / (
                SUM(
                    (CASE 
                        WHEN strftime('%s', lp."End Time") < strftime('%s', lp."Start Time")
                        THEN strftime('%s', lp."End Time") + 24*60*60
                        ELSE strftime('%s', lp."End Time")
                    END) - strftime('%s', lp."Start Time")
                ) / 3600.0
            ), 
        2) AS batch_speed,
        ROUND(SUM(
            IFNULL(ld.Factor_1, 0) +
            IFNULL(ld.Factor_2, 0) +
            IFNULL(ld.Factor_3, 0) +
            IFNULL(ld.Factor_4, 0) +
            IFNULL(ld.Factor_5, 0) +
            IFNULL(ld.Factor_6, 0) +
            IFNULL(ld.Factor_7, 0) +
            IFNULL(ld.Factor_8, 0) +
            IFNULL(ld.Factor_9, 0) +
            IFNULL(ld.Factor_10, 0) +
            IFNULL(ld.Factor_11, 0) +
            IFNULL(ld.Factor_12, 0)
        ) / 60, 2) AS total_downtime_hours,
        ROUND(
            SUM(
                IFNULL(ld.Factor_2, 0) +
                IFNULL(ld.Factor_5, 0) +
                IFNULL(ld.Factor_6, 0) +
                IFNULL(ld.Factor_8, 0) +
                IFNULL(ld.Factor_10, 0) +
                IFNULL(ld.Factor_11, 0)
            ) / 60, 2
        ) AS operator_downtime_hours
    FROM Line_downtime ld 
    LEFT JOIN line_productivity lp
        ON lp.Batch = ld.Batch
    GROUP BY lp.Operator
    ORDER BY lp.Operator
)
SELECT
    Operator,
    Batches_completed,
    actual_time_taken_hours,
    batch_speed,
    total_downtime_hours,
    operator_downtime_hours,
    round((operator_downtime_hours / actual_time_taken_hours) * 100,2) AS operator_error
FROM operator_performance_analysis;


 * sqlite:///manufacturing.db
Done.


Operator,Batches_completed,actual_time_taken_hours,batch_speed,total_downtime_hours,operator_downtime_hours,operator_error
Charlie,11,19.3,0.57,6.4,3.8,19.69
Dee,11,17.17,0.64,6.17,3.2,18.64
Dennis,8,13.67,0.59,5.03,2.73,19.97
Mac,8,14.17,0.56,5.53,3.2,22.58


if we regard speed more than accuracy ,the ranking will be:

| RANK     | OPERATOR | SPEED     | OPERATOR ERROR | 
|----------|----------|-----------|----------------|
| 1        | Dee      | 0.64      | 18.64          |
| 2        | Dennis   | 0.59      | 19.97          |
| 3        | Charlie  | 0.57      | 19.69          |
| 4        | Mac      | 0.56      | 22.58          |

However if we regard accuracy more than speed then the ranking will be:

| RANK     | OPERATOR | SPEED     | OPERATOR ERROR | 
|----------|----------|-----------|----------------|
| 1        | Dee      | 0.64      | 18.64          |
| 2        | Charlie  | 0.57      | 19.69          |
| 3        | Dennis   | 0.59      | 19.96          |
| 4        | Mac      | 0.56      | 22.58          |